In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os


In [ ]:
sns.set(style='darkgrid')


In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sample = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')


In [ ]:
train['sex'].fillna('unknown', inplace=True)
test['sex'].fillna('unknown', inplace=True)

train['age_approx'].fillna(train['age_approx'].mode().values[0], inplace=True)
test['age_approx'].fillna(test['age_approx'].mode().values[0], inplace=True)

train['anatom_site_general_challenge'].fillna('unknown', inplace=True)
test['anatom_site_general_challenge'].fillna('unknown', inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

train['sex_enc'] = enc.fit_transform(train.sex.astype('str'))
test['sex_enc'] = enc.transform(test.sex.astype('str'))

train['age_enc'] = enc.fit_transform(train.age_approx.astype('str'))
test['age_enc'] = enc.transform(test.age_approx.astype('str'))

train['anatom_enc'] = enc.fit_transform(train.anatom_site_general_challenge.astype('str'))
test['anatom_enc'] = enc.transform(test.anatom_site_general_challenge.astype('str'))


In [ ]:
train.head(5)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import roc_auc_score


In [ ]:
train.columns


In [ ]:
features = [
            'sex_enc',
            'age_enc',
            'anatom_enc'
]


In [ ]:
X = train[features]
y = train['target']

x_test = test[features]


In [ ]:
model = XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
                    importance_type='gain', interaction_constraints=None,
                     min_child_weight=1, missing=None, monotone_constraints=None,
                     n_estimators=700, n_jobs=-1, nthread=-1, num_parallel_tree=1
                    )

kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(model, X, y, cv=kfold, scoring='roc_auc', verbose=3)
cv_results.mean()


In [ ]:
xgb =XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
                    importance_type='gain', interaction_constraints=None,
                     min_child_weight=1, missing=None, monotone_constraints=None,
                     n_estimators=700, n_jobs=-1, nthread=-1, num_parallel_tree=1
                    )
xgb.fit(X, y)
pred = xgb.predict(x_test)


In [ ]:
sub = pd.DataFrame({'image_name':test.image_name.values,
                    'target':pred})
sub.to_csv('submission.csv',index = False)


In [ ]:
sub.head()
